In [120]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import recall_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import GridSearchCV

from xgboost import XGBRegressor

In [214]:
df=pd.read_csv('DATA/house_price/train.csv',usecols= lambda x: x!='Id')
test_x= pd.read_csv('DATA/house_price/test.csv')
sub= pd.read_csv('DATA/house_price/sample_submission.csv')

In [197]:
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',100)
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [215]:
def prep(df,test_x):
    global pip,grid
    df.rename(columns= str.lower,inplace=True)
    test_x.rename(columns= str.lower,inplace=True)
    # As we see till 'fence' our values mostly is missing
    # So, we can remove them permanently
    def prep_t(dff):
        dff['year'] = dff['yrsold'] - dff['yearbuilt']
        dff.drop(labels= ['yearremodadd', 'garageyrblt'],axis=1,inplace=True)
        dff['totrmsabvgrd']= (dff['totrmsabvgrd'] - dff['totrmsabvgrd'].max()) /(dff.totrmsabvgrd.max() -\
                                                                                 dff.totrmsabvgrd.min())
        dff['totrmsabvgrd']= np.power(dff['totrmsabvgrd'],2)
        dff['mssubclass']= np.power(dff['mssubclass'],2)
        dff['garagearea']= np.power(dff['garagearea'],2)
        dff['lotarea']= [np.log(i) if i!=0 else 0 for i in dff['lotarea'].values]

        ln_cols= ['masvnrarea', 'bsmtfinsf1', 'totalbsmtsf', 'openporchsf']
        dff[ln_cols] = np.array([[np.log(i) if i!=0 else 0 for i in dff[c].values] for c in ln_cols]).transpose()
        # creating new features

        high_corr= [('totalbsmtsf', '1stflrsf'),
                     ('grlivarea', 'totrmsabvgrd'),
                     ('garagecars', 'garagearea')]

        dff['ttlbs_1stflr']= dff[high_corr[0][0]]*dff[high_corr[0][1]]
        dff['grli_totrm']= dff[high_corr[1][0]]*dff[high_corr[1][1]]
        dff['grg_car_grg_area']= dff[high_corr[2][0]]*dff[high_corr[2][1]]
        return dff
    df=prep_t(df)

    high_null= (df.isnull().sum()*100/len(df)).sort_values(ascending=False)[:4]
    df.drop(labels=high_null.index,axis=1,inplace=True)
    
    
    
    
    
    x=df.drop(labels='saleprice',axis=1)
  
    y=df['saleprice']
    
    x['masvnrtype'].replace('None',np.NaN,inplace=True)
    columns_for_imputing= x.loc[:,x.isnull().sum()>10].isnull().sum().index
    col_for_imp_num= df[columns_for_imputing].select_dtypes(include=['int64','float64']).columns
    col_for_imp_cat= df[columns_for_imputing].select_dtypes(include=["O"]).columns
    
    for i in col_for_imp_num:
        test= x.loc[x[i].isnull(),i].index
        train= x.loc[x[i].notnull(),i].index
        y_ = x[i]
        x_ = x.drop(i,axis=1)
        x_= pd.get_dummies(x_.select_dtypes('O'))
        nul_x_train,nul_x_test,nul_y_train,nul_y_test= x_.loc[train],x_.loc[test],y_.loc[train],y_.loc[test ]
        model= DecisionTreeRegressor()
        model.fit(nul_x_train,nul_y_train)
        print(mse(nul_y_train,model.predict(nul_x_train)))
        print(mse(nul_y_train,[nul_y_train.mean()]*len(nul_x_train)))
        # I think result is good enough
        x.loc[test,i]= model.predict(nul_x_test)
    for i in col_for_imp_cat:
        test= x.loc[x[i].isnull(),i].index
        train= x.loc[x[i].notnull(),i].index
        y_ = x[i]
        x_ = x.drop(i,axis=1)
        x_= pd.get_dummies(x_.select_dtypes('O'))
        nul_x_train,nul_x_test,nul_y_train,nul_y_test= x_.loc[train],x_.loc[test],y_.loc[train],y_.loc[test ]
        model= DecisionTreeClassifier()
        model.fit(nul_x_train,nul_y_train)
        print(recall_score(nul_y_train,model.predict(nul_x_train),average='macro'))
        print(recall_score(nul_y_train,[nul_y_train.mode()]*len(nul_x_train),average='macro'))
        # I think result is good enough
        x.loc[test,i]= model.predict(nul_x_test)
    c= x.loc[:,x.isnull().any()].columns
    num= x[c].select_dtypes(include=['int64','float64']).columns
    for i in num:
        x[i].fillna(x[i].mean(),inplace=True)
    x['electrical']= x['electrical'].fillna(x['electrical'].mode()[0])
    # LassoCV ile tapilan deyerlerin yerlesdirilmesi
    
    # tdm
                                                                        

    col_tr= ColumnTransformer(transformers=[
        ('num',StandardScaler(),x.select_dtypes(include= ['int64','float64']).columns)
    ])
    
    pip= Pipeline(steps=[
        ('scale',col_tr),
        ('model',XGBRegressor())
    ])
    param_space= {"model__max_depth":[i for i in range(1,11,2)],
                 "model__eta":[0.7,0.5,0.3,0.1,0.01],
                 "model__colsample_bytree":[0.3,0.5,0.7,0.9,1.0]}
    grid= GridSearchCV(pip,param_space,cv=10)
    test_x= prep_t(test_x)
    test_x= test_x.replace(np.log(0),0)
    x=x.replace(np.log(0))
    x=pd.get_dummies(x)
    test_x= pd.get_dummies(test_x)
    x_train,x_test,y_train,y_test= train_test_split(x,y)
    main_test= pd.DataFrame(index= test_x.index,columns= x.columns)
    inter= list(set(test_x.columns).intersection(set(x.columns)))
    dif_train= list(set(x.columns).difference(set(test_x.columns)))
    
    main_test[inter]= test_x[inter]
    
    main_test[dif_train]= 0 
    
    # There is some NaN value which cause to error
    ask= main_test.isnull().any()
    for i in range(len(ask)):
        if ask.values[i]==True:
            ret= ask.index[i]
            if main_test[ret].dtypes=='O' or main_test[ret].dtypes==str:
                main_test[ret].fillna(main_test[ret].mode(),inplace=True)
            else:
                main_test[ret].fillna(main_test[ret].mean(),inplace=True)


    grid.fit(x_train,y_train)
    print("**Train")
    print("Model")
    display(np.sqrt(mse(y_train,grid.predict(x_train))))
    print("Mean pro")
    display(np.sqrt(mse(y,[y.mean()]*len(y))))
    print("**Test")
    display(np.sqrt(mse(y_test,grid.predict(x_test))))
    return grid.predict(main_test)

In [216]:
model= prep(df,test_x)

0.31265611990008324
589.2581203146698
0.9921875
0.3333333333333333
0.9996147919876733
0.25
0.9904075964716698
0.25
0.9821219457104955
0.25
0.9909726748160042
0.16666666666666666
0.9998673036093418
0.16666666666666666
0.9908571676067893
0.2
0.9981060606060606
0.16666666666666666
0.9925789563014896
0.3333333333333333
1.0
0.2
1.0
0.2


c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:96: RuntimeWarning: divide by zero encountered in log
c:\users\acer\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:97: RuntimeWarning: divide by zero encountered in log


**Train
Model


10533.495376937039

Mean pro


79415.29188606751

**Test


28136.20539729316

In [ ]:
"""
MSE scores
**Train Model
    10533.495376937039
Mean pro
    79415.29188606751
**Test
    28136.20539729316
"""

In [220]:
subm= pd.DataFrame({"Id":test_x['id'],"SalePrice":model})

In [221]:
subm.head()

,Id,SalePrice
0,1461,125015.515625
1,1462,160162.750000
2,1463,182651.046875
3,1464,192237.750000
4,1465,205031.218750


In [222]:
subm.to_csv("house_submission6th.csv",index=False)